# 5.1.1 自定义块
自定义Module，实现：
    1. 模型结构定义(__init__)
    2. 前向传播(forward)
    3. 后向传播(由系统创建)

In [13]:
import torch
from torch.nn import functional as F

class MLP(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.hidden = torch.nn.Linear(20, 256)  # 隐藏层
        self.out = torch.nn.Linear(256, 10)     # 输出层
    def forward(self, x):
        x = F.relu(self.hidden(x))
        return self.out(x)
x = torch.randn(2, 20)
net = MLP()
net(x)

tensor([[-0.1050,  0.2428, -0.0579, -0.1521,  0.0568,  0.1470,  0.2087, -0.5661,
         -0.0411, -0.0514],
        [-0.1558, -0.3168, -0.0300,  0.1123, -0.0206,  0.4302, -0.2784, -0.3488,
          0.0053, -0.1011]], grad_fn=<AddmmBackward0>)

# 5.1.2 顺序块
自定义Sequential，实现：
    1. 将Module逐个追加函数
    2. 前向传播

In [14]:
# 注意继承自Module

class MySequential(torch.nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        for idx, module in enumerate(args):
            # 这里的_modules为OrderDict
            self._modules[str(idx)] = module
    
    def forward(self, x):
        # OrderDict保证按天降顺序遍历
        for module in self._modules.values():
            x = module(x)
        return x

net = MySequential(torch.nn.Linear(20, 256),torch.nn.Linear(256, 10))
net(x)

tensor([[ 0.6376, -0.3519, -0.5611, -0.5517,  0.0475, -0.3580,  0.0307, -0.0160,
         -0.0321,  0.2081],
        [-0.3124, -0.1186,  0.7636,  0.7608,  0.2044, -0.0019, -0.3058, -0.1946,
          0.5620,  0.0590]], grad_fn=<AddmmBackward0>)

# 5.1.3 在前向传播函数中执行代码
以下操作不会常用于代码，只是展示操作

In [15]:
class FixedHiddenMLP(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = torch.nn.Linear(20, 20)
    def forward(self, x):
        x = self.linear(x)
        x = F.relu(x @ self.rand_weight + 1)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()
net = FixedHiddenMLP()
net(x)

tensor(0.2673, grad_fn=<SumBackward0>)

Module可以混搭，大型网络中会常用

In [16]:
class NestMLP(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(20, 64), torch.nn.ReLU(),
            torch.nn.Linear(64, 32), torch.nn.ReLU()
        )
        self.linear = torch.nn.Linear(32, 16)
    def forward(self, x):
        return self.linear(self.net(x))
chimera = torch.nn.Sequential(NestMLP(), torch.nn.Linear(16, 20), FixedHiddenMLP())
chimera(x)

tensor(-0.2931, grad_fn=<SumBackward0>)